## Imports

In [ ]:
import torch

import networkx as nx

from sklearn.model_selection import train_test_split

In [ ]:
from utils.corpus_utils import Corpus, CorpusMaker

## Data opening

In [ ]:
filepath = 'data/Bunin-Temnye_allei.txt'

In [ ]:
with open(filepath, 'r', encoding='cp1251') as f:
    corpus_raw = f.read().lower()

In [ ]:
test_corpus_raw = corpus_raw.split('\n')[57: 56+100]

## Texts to graph

In [ ]:
corpus_maker = CorpusMaker()

In [ ]:
%%time
train_graph = corpus_maker.create_graph('lenta')

In [ ]:
test_graph = corpus_maker.create_graph(test_corpus_raw, dicts=(train_graph.token2idx, train_graph.idx2token))

## Saving

In [ ]:
torch.save(train_graph, 'data/train_oldtorch.pickle')
torch.save(test_graph, 'data/test_oldtorch.pickle')

In [ ]:
## Vizualization

In [ ]:
# G = nx.Graph(train_graph['edge_index'].tolist())

In [ ]:
# nx.write_gexf(G, "test.gexf")

In [ ]:
# nx.draw_networkx(G)